In [100]:
import numpy as np
import numpy.random as rand
from tqdm import tqdm_notebook as tqdm

In [79]:
def DSBM(k, n, p, q, F, random_state=None, Herm=True):
    if not (random_state == None):
        rand.seed(random_state)
    assert (F.shape == k, k) and (k >= 2) and (abs(p-0.5) <= 0.5) and (abs(q-0.5) <= 0.5)
    assert np.all(F + F.T == 1)
    N = k * n
    res = np.zeros((N,N), dtype=complex)
    for c in range(k):
        for d in range(c,k):
            prob = p if c == d else q
            cd_ns = rand.rand(n,n) <= prob
            dirs = 2 * (rand.rand(n,n) <= F[c,d]) - 1
            cd_ns = cd_ns * dirs
            res[c * n : (c+1) * n, d * n : (d+1) * n] = cd_ns
            
    res = (np.triu(res) - np.triu(res).T) * 1j
    
    if not Herm:
        res = 1 * (res/1j > 0)
    return res

In [91]:
F = np.array([
        [0.5, 0.25, 0.5, 0.75],
        [0.75, 0.5, 0.25, 0.5],
        [0.5, 0.75, 0.5, 0.25],
        [0.25, 0.5, 0.75, 0.5]
    ])
k, n = 4, 2500
p, q = 0.025, 0.025

In [92]:
N = k*n
P = np.ones(k) / k
C  = np.array([
    [0.5, 0.2, 0.2, 0.1],
    [0.4, 0.5, 0.05, 0.05],
    [0.1, 0.2, 0.5, 0.2],
    [0.1, 0.2, 0.3, 0.4]
])
c = int(n/2 * (p+q))

In [174]:
def DSBM_PA(k, N, P, C, c, a=2, initial_nodes_per_cluster=4, Herm=False, random_state=None):
    if not (random_state == None):
        rand.seed(random_state)
    flatten = lambda edges : np.array([e for es in edges for e in es]) if len(edges) > 0 else []
    def select(v, P):
        index = -1
        while(v > 0):
            index +=1
            v -= P[index]
        return max(0, index)
    
    import networkx as nx
    
    communities = list(map(list,np.reshape(np.arange(0,initial_nodes_per_cluster*k), (k,initial_nodes_per_cluster))))
    community_edges = [[]]*k

    G = nx.DiGraph()
    for i, com in enumerate(communities):
        G.add_nodes_from(com, community=i)

    for i, com in enumerate(communities):
        edges = flatten([[(com[j], c) for c in com[j+1:]] for j in range(len(com))])
        G.add_edges_from(edges)
        community_edges[i].extend(edges)

    for i, com in enumerate(communities[:-1]):
        G.add_edge(com[-1], communities[i+1][0])
        community_edges[i].append((com[-1],communities[i+1][0]))
        
    for i in tqdm(range(initial_nodes_per_cluster*k,N)):
        r = rand.random()
        comm = select(r, P)
        communities[comm].append(i)
        G.add_node(i,community=comm)
        for _ in range(c):
            dest_node = i 
            r2 = rand.random()
            dest = select(r2, C[comm])
            while dest_node == i:
                if rand.random() < c / (c+a):
                    edges = community_edges[comm]
                    _, dest_node = edges[rand.randint(len(edges))]
                else:
                    dest_node = rand.choice(communities[dest])
            G.add_edge(i, dest_node)
            community_edges[comm].append((i,dest_node))   
    prec_nodes = list(map(lambda i: len(flatten(communities[:i])), range(len(communities))))
    new_labels = dict(map(lambda p: p[::-1], enumerate(flatten(communities))))
    nx.relabel_nodes(G, lambda x: new_labels[x] * 'a', copy=False)
    print(list(G.nodes())[:100])
    res = nx.to_numpy_array(G, dtype=complex, nodelist=sorted(G.nodes()))
    if Herm:
        res = (res - res.T) * 1j
    return (res, new_labels, G, communities)

In [175]:
A, dic, H, comms = DSBM_PA(k, N, P, C, c, Herm=True)


['', 'a', 'aa', 'aaa', '...', 'aaaa', '...', 'aaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [74]:
import networkx as nx
tmp = nx.to_numpy_matrix(_)
tmp - tmp.T

matrix([[ 0.,  1.,  1., ...,  0.,  0., -1.],
        [-1.,  0.,  1., ...,  0.,  0.,  0.],
        [-1., -1.,  0., ..., -1., -1., -1.],
        ...,
        [ 0.,  0.,  1., ...,  0., -1.,  0.],
        [ 0.,  0.,  1., ...,  1.,  0.,  0.],
        [ 1.,  0.,  1., ...,  0.,  0.,  0.]])

In [61]:
C

array([[0.9  , 0.025],
       [0.075, 0.9  ]])

In [84]:
DSBM(k,n,p,q,F,random_state=28,Herm=False)

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 1, 0]])

In [93]:
c

62

In [110]:
G = nx.DiGraph()
G.add_nodes_from([1,2])
nx.relabel_nodes(G, {1:4, 2:3}, copy=True)

In [142]:
list(G.nodes())

[1, 2]

In [113]:
lambda : 2

SyntaxError: invalid syntax (<ipython-input-113-11d8914f3d49>, line 1)

In [125]:
flatten = lambda edges : np.array([e for es in edges for e in es]) if len(edges) > 0 else []
communities = list(map(list,np.reshape(rand.permutation(np.arange(0,4*2)), (2,4))))
print(communities)
dict(map(lambda p: p[::-1], enumerate(flatten(communities))))


[[6, 3, 1, 7], [0, 4, 5, 2]]


{6: 0, 3: 1, 1: 2, 7: 3, 0: 4, 4: 5, 5: 6, 2: 7}

In [121]:
rand.permutation([1,2,3])

array([1, 3, 2])

In [135]:
sorted(['a', 'aaa', 'aa'])

['a', 'aa', 'aaa']

In [166]:
dic[]

2457

In [172]:
np.imag(A[:10, :10])

array([[ 0.,  1.,  1.,  1.,  0., -1.,  0., -1., -1.,  0.],
       [-1.,  0.,  1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1.,  0.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1.,  0., -1., -1., -1., -1., -1., -1.],
       [ 0.,  1.,  1.,  1.,  0.,  0., -1., -1.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [187]:
comms[0][:10], dic[40]

([0, 1, 2, 3, 18, 30, 34, 36, 37, 40], 9)

In [194]:
np.imag(A[:13, :13])

array([[ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  1., -1., -1., -1., -1., -1., -1.,  0., -1., -1.],
       [-1., -1.,  0.,  1., -1., -1., -1., -1., -1.,  0., -1., -1.,  0.],
       [-1., -1., -1.,  0., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]])

In [199]:
len(list(H.nodes())[10])

5038

In [204]:
sorted(list(H.neighbors('aaaa')))

['a',
 'aa',
 'aaa',
 '...']